#Sentiment analysis
Fine-Tuning BERT
ECE590 FINAL



In [1]:
import functools
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
torch.backends.cudnn.benchmark = True

import os
os.makedirs("llm_models", exist_ok=True)
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, BertModel
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install datasets #needed for dataset

## Load Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion")

train_x = dataset['train']['text']
train_y = dataset['train']['label']
test_x = dataset['test']['text']
test_y = dataset['test']['label']
val_x = dataset['validation']['text']
val_y = dataset['validation']['label']


In [ ]:
len(train_x), len(val_x), len(test_x)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:

max_len= 128
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(train_x,
                                              add_special_tokens=True,
                                              return_attention_mask=True,
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [8]:

X_val_encoded = tokenizer.batch_encode_plus(val_x,
                                              padding=True,
                                              add_special_tokens=True,
                                              return_attention_mask=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [9]:

X_test_encoded = tokenizer.batch_encode_plus(test_x,
                                              add_special_tokens=True,
                                              return_attention_mask=True,
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [ ]:
#Show example
k = 350
print('Training Comments -->>',train_x[k])
print('\nInput Ids -->>\n',X_train_encoded['input_ids'][k])
print('\nDecoded Ids -->>\n',tokenizer.decode(X_train_encoded['input_ids'][k]))
print('\nAttention Mask -->>\n',X_train_encoded['attention_mask'][k])
print('\nLabels -->>',train_y[k])

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',problem_type="multi_label_classification", num_labels=6)
optimizer = tf.keras.optimizers.SGD(.01) #learning_rate=5e-05)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [ ]:
#Train the model
history = model.fit(
	[X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
	pd.Series(train_y),
	validation_data=(
	[X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],pd.Series(val_y)),
	batch_size=32,
	epochs=3
)


In [ ]:
#Evaluate on test set
test_loss, test_accuracy = model.evaluate(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
    pd.Series(test_y)
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

In [ ]:
model.summary()

In [ ]:
preds = model.predict([X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
logits = preds.logits
preds = np.argmax(logits, axis=1)
print(preds.shape)

In [ ]:
## PLOT CONFUSION MATRIX
cm = confusion_matrix(test_y, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])
disp.plot()

In [ ]:
## PLOT LOSSES
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title("BERT: Cross Entropy Loss by Epoch")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")
plt.legend()
plt.show()